Required Libraries

In [1]:
# Importing required libraries
import warnings
warnings.filterwarnings("ignore")

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import pandas as pd

import numpy as np

AID Data Analysis

In [2]:
# Read the 2019 AID Data a-file 
aid_data = pd.read_csv('a2019.csv',encoding='UTF-8',skipinitialspace=True)

aid_data1 = aid_data

# Data Cleaning and structuring
aid_data1.columns= aid_data1.columns.str.strip()

# aid_data1 = aid_data1.dropna(subset=['c5'],how='all')
# Drop the last two columns (Extra empty columns exists at the end of the file)
aid_data1 = aid_data1.drop(aid_data1.columns[[-1,-2]],axis=1)

# Check the columns c5 for any character string (This column should not contain text)
fil = aid_data1['c5'].str.contains(r'^[a-zA-Z]')

aid_data1 = aid_data1[~fil]

# Resetting the Index Values
aid_data1.reset_index(drop=True,inplace=True)

# Unifying the date formats 
aid_data1['c9'] = pd.to_datetime(aid_data1['c9'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c205'] = pd.to_datetime(aid_data1['c205'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c206'] = pd.to_datetime(aid_data1['c206'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c207'] = pd.to_datetime(aid_data1['c207'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c208'] = pd.to_datetime(aid_data1['c208'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c210'] = pd.to_datetime(aid_data1['c210'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c229'] = pd.to_datetime(aid_data1['c229'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c230'] = pd.to_datetime(aid_data1['c230'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c10'] = pd.to_datetime(aid_data1['c10'].astype(str),format= '%H%M.0',errors='coerce').dt.strftime('%H:%M')

# Making some columns as integer type for further analysis(most of these are the ones which provide the information about the number of people injured or killed on the aircraft)
aid_data1[aid_data1.columns[79:95]] = aid_data1[aid_data1.columns[79:95]].fillna(0)

for col in aid_data1.columns[80:93].values:
    aid_data1[col] = aid_data1[col].astype('int64')

#replacing the columns that has zero to NAN 
aid_data1[['c149','c151','c145']] = aid_data1[['c149','c151','c145']].fillna(0).astype(int).astype(str).replace('0','NaN')

# Renaming important columns and adding suffix to other columns to distinguish the data of one report from another to understand better for further analysis
aid_data1.rename(columns={'c9': 'Date_Of_Occurrence', 'c22': 'N_Number'}, inplace=True)

aid_data1 = aid_data1.add_suffix('_aid').rename(columns={'Report_Type_aid': 'Report_Type', 'Date_Of_Occurrence_aid': 'Date_Of_Occurrence'})

aid_data1 = aid_data1.rename(columns={'c5_aid': 'Unique_Identifier'})

# Inserting a new column of report type to help further analysis
aid_data1.insert(0,'Report_Type','AID')
    
# aid_data1[aid_data1.columns[79:95]]

In [3]:
#Top 10 records in the dataset
aid_data1.head(10)

,Report_Type,Unique_Identifier,c1_aid,c2_aid,c3_aid,c4_aid,c6_aid,c7_aid,c8_aid,Date_Of_Occurrence,c10_aid,c75_aid,c132_aid,c134_aid,c136_aid,c138_aid,c139_aid,c140_aid,c141_aid,c144_aid,c145_aid,c147_aid,c149_aid,c151_aid,c152_aid,c153_aid,c155_aid,c157_aid,c160_aid,c162_aid,c203_aid,c204_aid,c214_aid,c233_aid,c234_aid,c790_aid,N_Number_aid,c23_aid,c24_aid,c25_aid,...,c192_aid,c244_aid,c40_aid,c44_aid,c46_aid,c48_aid,c51_aid,c77_aid,c79_aid,c81_aid,c83_aid,c85_aid,c87_aid,c89_aid,c91_aid,c93_aid,c95_aid,c97_aid,c99_aid,c101_aid,c103_aid,c105_aid,c107_aid,c109_aid,c131_aid,c133_aid,c135_aid,c137_aid,c146_aid,c148_aid,c150_aid,c154_aid,c156_aid,c158_aid,c161_aid,c163_aid,c183_aid,c191_aid,c229_aid,c230_aid
0,AID,20191013023229A,A,91,NaN,NaN,2019,10,13,2019-10-13,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CV,NaN,NaN,224306,2322,NaN,NaN,NaN,NaN,96709,TAYLOR,BC12-D,NaN,...,NaN,NaN,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takeoff: Climb Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wheeled-Conventional,NaN,NaN,NaN,NaN,1950-07-28,NaT
1,AID,20191122023189A,A,91,NaN,NaN,2019,11,22,2019-11-22,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224301,2318,NaN,NaN,NaN,NaN,969JM,CIRRUS,SR22T,NaN,...,NaN,NaN,COMM PILOT FLIGHT IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruise/Level Flight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1967-07-29,NaT
2,AID,20191125023199I,I,91,NaN,NaN,2019,11,25,2019-11-25,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224302,2319,NaN,NaN,NaN,NaN,3201L,MOONEY,M20J,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Other, Specify",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1961-12-21,NaT
3,AID,20191217023209A,A,91,NaN,NaN,2019,12,17,2019-12-17,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224303,2320,NaN,NaN,NaN,NaN,3113V,BEECH,35R,NaN,...,NaN,NaN,COMMERCIAL PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Taxi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1933-11-17,NaT
4,AID,20191218023219I,I,91,NaN,NaN,2019,12,18,2019-12-18,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR,NaN,NaN,224304,2321,NaN,NaN,NaN,NaN,6919J,PIPER,PA-28-151,NaN,...,NaN,NaN,PRIVATE PILOT FLT IN,NaN,NaN,NaN,NaN,6111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruise/Level Flight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,2000-02-19,2001-06-13
5,AID,20191101023269A,A,91,NaN,NaN,2019,11,1,2019-11-01,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1H71,1,H,7,1,3O,NaN,NaN,TR,NaN,NaN,224310,2326,NaN,NaN,NaN,NaN,102PH,HATFIE,CH701,NaN,...,NaN,NaN,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takeoff: Climb Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-HIGH WING/PARA WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1950-03-10,NaT
6,AID,20191101023289A,A,91,NaN,NaN,2019,11,1,2019-11-01,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1H71,1,H,7,1,3O,NaN,NaN,TR,NaN,NaN,224312,2328,NaN,NaN,NaN,NaN,732JW,CESSNA,T210L,NaN,...,NaN,NaN,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takeoff: Climb Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-HIGH WING/PARA WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1967-02-27,NaT
7,AID,20191031023279A,A,91,NaN,NaN,2019,10,31,2019-10-31,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L72,1,L,7,2,3O,NaN,NaN,TR,NaN,NaN,224311,2327,NaN,NaN,NaN,NaN,959CM,RAYTHE,58,NaN,...,NaN,NaN,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Landing: Approach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN

E-File AID Data Analysis

In [4]:
import more_itertools as mit
# Read the 2019 AID Data e-file 

e_aid_data = pd.read_csv('e2019(possible).csv',encoding='UTF-8',skipinitialspace=True)

e_aid_data1= e_aid_data

# Dropping blank rows from primary key column of c5
e_aid_data1 = e_aid_data1.dropna(subset=['c5'],how='all')

# Combining all the text data into one separate column and dropping the individual text columns
e_aid_data1['remark'] = e_aid_data1[['remark','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9']].replace('?','').fillna('').agg(' '.join, axis=1)
e_aid_data1 = e_aid_data1.drop(e_aid_data1.columns[[2,3,4,5,6,7,8,9]],axis=1)

e_aid_data1.reset_index(drop=True,inplace=True)

e_aid_data1['remark']= e_aid_data1['remark'].replace(r'^\s*$', np.nan, regex=True) #replacing extra whitespace
e_aid_data1['remark']= e_aid_data1['remark'].mask(pd.isnull, e_aid_data1['c5']) #

fill = e_aid_data1['c5'].str.contains(r'^\b2019')

e_aid_data1['c5'][fill==False] = np.nan

#Unifying all the related text remarks corresponding to each unique identifier of AID
q=[index for index, row in e_aid_data1.iterrows() if row.isnull().any()]

q=[list(group) for group in mit.consecutive_groups(q)] 

for i in q:
    t=''
    s=i[0]
    for j in i:
        t+=(e_aid_data1.iloc[j]['remark'])
        t+=' '
    e_aid_data1.iloc[s-1]['remark']+=t
        
e_aid_data1 = e_aid_data1[pd.notnull(e_aid_data1['c5'])]

e_aid_data1.reset_index(drop=True,inplace=True)

# Renaming the columns for better understanding
e_aid_data1.rename(columns={'c5':'c5_aid','remark': 'Remarks'},inplace=True)

e_aid_data1 = e_aid_data1.rename(columns={'c5_aid': 'Unique_Identifier'})

# Part of text preprocessing of remarks column
e_aid_data1['Remarks'] = e_aid_data1['Remarks'].str.replace('\^PRIVACY DATA OMITTED\^', '').str.strip().str.replace('\s+', ' ', regex=True)

e_aid_data1['Remarks'] = e_aid_data1['Remarks'].str.replace('(#NAME\?)', '').str.strip().str.replace('\s+', ' ', regex=True)

e_aid_data1.insert(0,'Report_Type','AID') #inserting a column which represents the type of report

e_aid_data1

,Report_Type,Unique_Identifier,Remarks
0,AID,20190101007219I,DURING APPROACH TO STUART WITHAM AIRPORT THE A...
1,AID,20190101007229I,THE ONLY INFORMATION PROVIDED BY AIRMAN WAS TH...
2,AID,20190101008169I,"ON DEPARTURE ROLL FROM PORT EADS GRASS STRIP, ..."
3,AID,20190101016219I,"VENICE, FL. N241EE, BE35 LANDED ON RUNWAY 23 A..."
4,AID,20190102000019A,1/2/19 AT 1045 EST. N772MG AN R44 HELICOPTER C...
...,...,...,...
2403,AID,20191231019739I,PILOT WAS IN THE TRAFFIC PATTERN FOR THE CREST...
2404,AID,20191231019749I,PILOT WAS CONDUCTING A FAR PART 91 TRIP TO MCC...
2405,AID,20191231019759A,PILOT UTILIZING PRIVATE MOWED STRIP TO PRACTIC...
2406,AID,20191231020669A,THE PILOT WITH 3 PASSENGERS INITIATED A TAKEOF...


In [5]:
# Merging of e-file of AID with the a-file of AID to have the full remarks column in one dataset
aid_data1 = pd.merge(aid_data1, e_aid_data1, how="inner", on=["Report_Type", "Unique_Identifier"])

# Dropping C119 column
aid_data1 = aid_data1.drop('c119_aid',axis=1)

#Replacing missing values in Phase of flight column with 'Unknown'
aid_data1['c95_aid'] = aid_data1['c95_aid'].fillna('Unknown')

#Replacing missing values in Aircraft Make column with 'Unknown'
aid_data1['c23_aid']= aid_data1['c23_aid'].fillna('Unknown')

#Replacing missing values in Aircraft Model column with 'Aircraft'
aid_data1['c24_aid']= aid_data1['c24_aid'].fillna('Aircraft')

#Concatenating Aircraft Make and Model to a single column called Aircraft Type
aid_data1['Aircraft_Type'] = aid_data1['c23_aid'] + ' ' + aid_data1['c24_aid']
aid_data1['Aircraft_Type'] = aid_data1['Aircraft_Type'].str.strip().str.replace('\s+', ' ', regex=True).astype(str)

#Renaming as per convenience
aid_data1 = aid_data1.rename(columns={'c95_aid':'Phase_Of_Flight','c23_aid':'Aircraft_Make'})

#Removing the extra spaces to the left and right of the data in Phase of Flight column
aid_data1['Phase_Of_Flight'] = aid_data1['Phase_Of_Flight'].str.strip()

#Top 10 rows in AID
aid_data1.head(10)

,Report_Type,Unique_Identifier,c1_aid,c2_aid,c3_aid,c4_aid,c6_aid,c7_aid,c8_aid,Date_Of_Occurrence,c10_aid,c75_aid,c132_aid,c134_aid,c136_aid,c138_aid,c139_aid,c140_aid,c141_aid,c144_aid,c145_aid,c147_aid,c149_aid,c151_aid,c152_aid,c153_aid,c155_aid,c157_aid,c160_aid,c162_aid,c203_aid,c204_aid,c214_aid,c233_aid,c234_aid,c790_aid,N_Number_aid,Aircraft_Make,c24_aid,c25_aid,...,c40_aid,c44_aid,c46_aid,c48_aid,c51_aid,c77_aid,c79_aid,c81_aid,c83_aid,c85_aid,c87_aid,c89_aid,c91_aid,c93_aid,Phase_Of_Flight,c97_aid,c99_aid,c101_aid,c103_aid,c105_aid,c107_aid,c109_aid,c131_aid,c133_aid,c135_aid,c137_aid,c146_aid,c148_aid,c150_aid,c154_aid,c156_aid,c158_aid,c161_aid,c163_aid,c183_aid,c191_aid,c229_aid,c230_aid,Remarks,Aircraft_Type
0,AID,20191013023229A,A,91,NaN,NaN,2019,10,13,2019-10-13,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CV,NaN,NaN,224306,2322,NaN,NaN,NaN,NaN,96709,TAYLOR,BC12-D,NaN,...,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takeoff: Climb Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wheeled-Conventional,NaN,NaN,NaN,NaN,1950-07-28,NaT,THE PILOT OBTAINED LOCAL AUTOMATED WEATHER REP...,TAYLOR BC12-D
1,AID,20191122023189A,A,91,NaN,NaN,2019,11,22,2019-11-22,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224301,2318,NaN,NaN,NaN,NaN,969JM,CIRRUS,SR22T,NaN,...,COMM PILOT FLIGHT IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruise/Level Flight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1967-07-29,NaT,DEPARTED TRL FOR FLIGHT TO GLADEWATER. CLIMBED...,CIRRUS SR22T
2,AID,20191125023199I,I,91,NaN,NaN,2019,11,25,2019-11-25,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224302,2319,NaN,NaN,NaN,NaN,3201L,MOONEY,M20J,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Other, Specify",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1961-12-21,NaT,"ON NOVEMBER 25, 2019, AT APPROXIMATELY 1130 EA...",MOONEY M20J
3,AID,20191217023209A,A,91,NaN,NaN,2019,12,17,2019-12-17,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224303,2320,NaN,NaN,NaN,NaN,3113V,BEECH,35R,NaN,...,COMMERCIAL PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Taxi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1933-11-17,NaT,THE PILOT/OWNER WAS INTENDING TO MAKE A SHORT ...,BEECH 35R
4,AID,20191218023219I,I,91,NaN,NaN,2019,12,18,2019-12-18,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR,NaN,NaN,224304,2321,NaN,NaN,NaN,NaN,6919J,PIPER,PA-28-151,NaN,...,PRIVATE PILOT FLT IN,NaN,NaN,NaN,NaN,6111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruise/Level Flight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,2000-02-19,2001-06-13,A PORTION OF THE PROPELLER SEPARATED FROM AIRC...,PIPER PA-28-151
5,AID,20191101023269A,A,91,NaN,NaN,2019,11,1,2019-11-01,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1H71,1,H,7,1,3O,NaN,NaN,TR,NaN,NaN,224310,2326,NaN,NaN,NaN,NaN,102PH,HATFIE,CH701,NaN,...,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takeoff: Climb Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-HIGH WING/PARA WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1950-03-10,NaT,"ON FRIDAY, NOVEMBER 1, 2019, A ZENITH CH701, R...",HATFIE CH701
6,AID,20191101023289A,A,91,NaN,NaN,2019,11,1,2019-11-01,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1H71,1,H,7,1,3O,NaN,NaN,TR,NaN,NaN,224312,2328,NaN,NaN,NaN,NaN,732JW,CESSNA,T210L,NaN,...,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takeoff: Climb Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-HIGH WING/PARA WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1967-02-27,NaT,"ON NOVEMBER 1, 2019, AT 1350 CDT, A CESS

In [6]:
#Dimensions of the Dataset
aid_data1.shape

(2391, 181)

SDR Data Analysis

In [7]:
# Read the 2019 SDR Data file 

sdr_data = pd.read_csv('sdr(2019_Occurred_Incidents).csv',encoding='UTF-8',skipinitialspace=True,dtype=str)
sdr_data1 = sdr_data

# Dropping blank rows (if any) from the main column of c25
sdr_data1 = sdr_data1.dropna(subset=['c25'],how='all')

# Dropping columns 
sdr_data1 = sdr_data1.drop(sdr_data1.columns[[-1,-2,-3,0]],axis=1)
#Removing of spaces
sdr_data1.columns= sdr_data1.columns.str.strip()

#changing the date format into Year/month/day
sdr_data1['c10'] = pd.to_datetime(sdr_data1['c10'].astype(str), format='%Y%m%d',errors='coerce')
sdr_data1['c25'] = pd.to_datetime(sdr_data1['c25'].astype(str), format='%Y%m%d',errors='coerce')

# Joining the Remarks columns
sdr_data1['Remarks'] = sdr_data1[['c510a','c510b','c510c','c510d','c510e','c510f']].fillna('').agg(''.join, axis=1)

# Removing extra whitespace
sdr_data1['Remarks']= sdr_data1['Remarks'].str.strip().str.replace('\s+', ' ', regex=True)

#dropping the duplicate columns 
sdr_data1.drop(['c510a', 'c510b','c510c','c510d','c510e','c510f'], axis=1, inplace=True)

#Renaming the columns
sdr_data1.rename(columns={'c25': 'Date_Of_Occurrence', 'c390': 'N_Number'}, inplace=True)

#Adding sdr as suffix
sdr_data1 = sdr_data1.add_suffix('_sdr').rename(columns={'Report_Type_sdr': 'Report_Type', 'Date_Of_Occurrence_sdr': 'Date_Of_Occurrence','N_Number_sdr':'N_Number','Remarks_sdr':'Remarks'})

sdr_data1['c18_sdr'] = sdr_data1['c18_sdr'].str.replace('\W', '')

#Renaming the c18 column as Unique Identifier
sdr_data1 = sdr_data1.rename(columns={'c18_sdr': 'Unique_Identifier'})

#Filling null values of Aircraft Make column with 'Unknown'
sdr_data1['c130_sdr']=sdr_data1['c130_sdr'].fillna('Unknown')

#Filling null values of Aircraft Model column with 'Unknown'
sdr_data1['c140_sdr']=sdr_data1['c140_sdr'].fillna('Aircraft')

sdr_data1['c100_sdr'] = sdr_data1['c100_sdr'].str.strip()

sdr_data1['c250_sdr'] = sdr_data1['c250_sdr'].str.strip()

sdr_data1['c260_sdr'] = sdr_data1['c260_sdr'].str.strip()

sdr_data1['c100_sdr']=sdr_data1['c100_sdr'].fillna('UNKNOWN')

sdr_data1['c250_sdr']=sdr_data1['c250_sdr'].fillna('UNKNOWN')

sdr_data1['c260_sdr']=sdr_data1['c260_sdr'].fillna('UNKNOWN')

sdr_data1['Aircraft_Type'] = sdr_data1['c130_sdr'] + ' '+ sdr_data1['c140_sdr']
sdr_data1['Aircraft_Type'] = sdr_data1['Aircraft_Type'].str.strip().str.replace('\s+', ' ', regex=True).astype(str)

sdr_data1['Remarks'] = sdr_data1['Remarks'].str.replace('\x1a', '')

sdr_data1.rename(columns={'c332_sdr':'Stage_Of_Operation','c100_sdr':'Description_Of_Defective_Part','c250_sdr':'Location_Of_Defective_Part','c260_sdr':'Condition_Of_Defective_Part','c130_sdr':'Aircraft_Make'}, inplace=True)

sdr_data1['Stage_Of_Operation'] = sdr_data1['Stage_Of_Operation'].str.strip()


sdr_data1.insert(0,'Report_Type','SDR')

# print the first 5 rows
sdr_data1.head(5)

,Report_Type,c10_sdr,c12_sdr,c14_sdr,c15_sdr,c16_sdr,c17_sdr,Unique_Identifier,c20_sdr,Date_Of_Occurrence,c35_sdr,c40_sdr,c90_sdr,Description_Of_Defective_Part,c110_sdr,c120_sdr,Aircraft_Make,c140_sdr,c150_sdr,c152_sdr,c160_sdr,c170_sdr,c180_sdr,c190_sdr,c192_sdr,c200_sdr,c210_sdr,c220_sdr,c230_sdr,c240_sdr,Location_Of_Defective_Part,Condition_Of_Defective_Part,c270_sdr,c280_sdr,c290_sdr,c300_sdr,c310a_sdr,c310b_sdr,c310c_sdr,c310d_sdr,...,c314b_sdr,c314c_sdr,c314d_sdr,c320a_sdr,c320b_sdr,c320c_sdr,c324a_sdr,c324b_sdr,c324c_sdr,c330_sdr,Stage_Of_Operation,c340_sdr,c350_sdr,c360_sdr,c370_sdr,c380_sdr,N_Number,c400_sdr,c410_sdr,c420_sdr,c430_sdr,c440_sdr,c450_sdr,c460_sdr,c490_sdr,c602_sdr,c604_sdr,c606_sdr,c608_sdr,c610_sdr,c612_sdr,c614_sdr,c616_sdr,c620_sdr,c640_sdr,c652_sdr,c654_sdr,c660_sdr,Remarks,Aircraft_Type
0,SDR,2019-09-06,NaN,NM,2019,9,6,FDEA201904040764,1,2019-04-04,A,5320,ACA314011,FLOOR SUPPORT,NaN,NaN,DOUG,MD11,MD11F,3023600,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZONE 200,CORRODED,A,NaN,NaN,FDEA,K,NaN,NaN,NaN,...,NaN,NaN,NaN,O,NaN,NaN,OTHER,NaN,NaN,IN,INSP/MAINT,1,NaN,NaN,CE,23,604FE,NaN,NaN,NaN,NaN,48460,NaN,NaN,NaN,NaN,NaN,NaN,3,4F,NaN,NaN,RT,A22WE,NaN,NaN,NaN,NaN,FLOOR BOARD SUPPORT CORRODED @ STA. 6-70 CENTE...,DOUG MD11
1,SDR,2019-11-18,NaN,NE,2019,11,18,11FA2019070100001,2,2019-05-23,A,7200,NaN,ENGINE,NaN,NaN,CESSNA,510,510,2076617,CE,PWC,PW615,PW615FA,52282,NE,NaN,NaN,NaN,NaN,NR 2,MALFUNCTIONED,E,NaN,NaN,11FA,C,NaN,NaN,NaN,...,NaN,NaN,NaN,M,J,NaN,OVER TEMP,WARNING INDICATION,NaN,TO,TAKEOFF,1,NaN,NaN,SW,3,820UT,NaN,NaN,NaN,NaN,5100332,LB0666,NaN,NaN,NaN,NaN,NaN,2,4F,4,F,RT,A00014WI,E00073EN,NaN,NaN,NaN,ITT SPIKE NR 2 ENGINE EXCEEDED REDLINE. ENGINE...,CESSNA 510
2,SDR,2019-11-18,NaN,CE,2019,11,18,11FA2019070100002,1,2019-05-20,A,7722,NaN,UNKNOWN,NaN,NaN,CESSNA,510,510,2076617,CE,PWC,PW615,PW615FA,52282,NE,NaN,NaN,NaN,NaN,UNKNOWN,MALFUNCTIONED,E,NaN,NaN,11FA,K,NaN,NaN,NaN,...,NaN,NaN,NaN,J,M,NaN,WARNING INDICATION,OVER TEMP,NaN,CL,CLIMB,1,NaN,NaN,SW,3,820UT,NaN,NaN,NaN,NaN,5100332,LB0666,NaN,NaN,NaN,NaN,NaN,2,4F,4,F,RT,A00014WI,E00073EN,NaN,NaN,NaN,ITT SPIKE IN THE CLIMB FOR UNKNOWN REASONS. MA...,CESSNA 510
3,SDR,2020-03-24,NaN,NM,2020,3,24,120B201911200001,1,2019-11-18,A,2697,5534001R16,WIRE HARNESS,KIDDE,NA,BOEING,767,767323,1385205,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RT FIRE LOOP,BROKEN,I,NaN,NaN,120B,K,NaN,NaN,NaN,...,NaN,NaN,NaN,O,NaN,NaN,OTHER,NaN,NaN,IN,INSP/MAINT,1,NaN,NaN,GL,3,36NE,NaN,NaN,NaN,NSN,25193,NaN,NaN,NaN,NaN,NaN,NaN,2,4F,NaN,NaN,RT,A1NM,NaN,NaN,NaN,NaN,RH ENGINE FIRE LOOP 1 DISPLAYED ON EICAS ON PR...,BOEING 767
4,SDR,2020-06-02,NaN,NM,2020,6,2,120B201912240002,1,2019-12-23,A,2841,2021207,INDICATOR,NaN,NaN,BOEING,767,767323,1385205,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FUEL QTY,INOPERATIVE,I,NaN,NaN,120B,H,NaN,NaN,NaN,...,NaN,NaN,NaN,J,NaN,NaN,WARNING INDICATION,NaN,NaN,NR,NOT REPORTED,1,NaN,NaN,GL,3,225NE,99331,16,NaN,NaN,25194,NaN,NaN,NaN,NaN,NaN,NaN,2,4F,NaN,NaN,RT,A1NM,NaN,NaN,NaN,NaN,FUEL QUANTITY SYSTEM DISPLAYED ON PREFLIGHT EI...,BOEING 767


In [9]:
#Dimensions of the Dataset
sdr_data1.shape

(61009, 81)

EON Data Analysis

In [8]:
# Read the 2019 EON Data file 

eon_data = pd.read_csv('EON-2019.csv',encoding='UTF-8',skipinitialspace=True)
eon_data1 = eon_data

# eon_data1 = eon_data1.dropna(axis='columns', how='all')
#Removing the spaces
eon_data1.columns= eon_data1.columns.str.strip()

#Changing the event date to date and time format 
eon_data1['Event Date'] = pd.to_datetime(eon_data1['Event Date'])
eon_data1['Event Time'] = eon_data1['Event Date'].dt.strftime('%H:%M')
eon_data1['Event Date'] = eon_data1['Event Date'].dt.strftime('%Y-%m-%d')

eon_data1['Event Date'] = pd.to_datetime(eon_data1['Event Date'])

cols = eon_data1.columns.tolist()
cols = cols[0:5]+cols[-1:]+cols[5:36]
eon_data1 = eon_data1[cols]

# joining the Remarks column  
eon_data1['Message Whole'] = eon_data1[['MessageText.1','Unnamed: 29','Unnamed: 30','Unnamed: 31','Unnamed: 32','Unnamed: 33','Unnamed: 34','Unnamed: 35']].fillna('').agg(''.join, axis=1)

eon_data1['Message Whole'] = eon_data1['Message Whole'].str.strip().str.replace('\s+', ' ', regex=True)

#renaming the columns
eon_data1.rename(columns={'Message Whole': 'Remarks','Event Date': 'Date_Of_Occurrence'},inplace=True)

cols1 = eon_data1.columns.tolist()
cols1 = cols1[0:29] + cols1[-1:]

eon_data1 = eon_data1[cols1]

#Extracting the possible N-Number from Title column and made it as a seperate column
eon_data1['N_Number'] = eon_data1['Title'].str.extract(r'((?<=\|).+?(?=\|))')

#Removing the spaces
eon_data1['N_Number'] = eon_data1['N_Number'].str.strip()

fil = eon_data1['N_Number'].str.contains('^N[0-9]|^n[0-9]')

# Replacing the 'n' to N
eon_data1['N_Number'][fil==True] = eon_data1['N_Number'][fil==True].apply(lambda x : x[1:] if str(x).startswith("N") else x).apply(lambda x : x[1:] if str(x).startswith("n") else x)

#Renaming ID column with Unique Identifier
eon_data1 = eon_data1.rename(columns={'Id': 'Unique_Identifier'})

#Removing Left spaces in the remarks column
eon_data1.Remarks = eon_data1.Remarks.str.lstrip(',').str.strip()
#Dropping of Duplicates in Unique Identifier
eon_data1 = eon_data1.drop_duplicates(subset=['Unique_Identifier'])
#Resetting of the index
eon_data1.reset_index(drop=True,inplace=True)

# Checking whether Aircraft type is present in the remarks if present then Remove the aircraft type from the Remarks
eon_data1['Aircraft Type'][eon_data1['Aircraft Type'].isna()] = ''
for i in range(len(eon_data1)):
  temp = eon_data1.loc[i,'Aircraft Type']
  if temp in eon_data1.loc[i,'Remarks']:
    eon_data1.loc[i,'Remarks'] = eon_data1.loc[i,'Remarks'].replace(temp,'')
    eon_data1.loc[i,'Remarks'] = eon_data1.loc[i,'Remarks'].replace(',','').strip()

#Blank Remarks is replaced with Not Reported
eon_data1.Remarks[eon_data1.Remarks == ''] = 'NOT REPORTED'

#Replacing Blank and Unknown values in Aircraft Type column to hold a single string 'UNKNOWN'
eon_data1['Aircraft Type'][eon_data1['Aircraft Type']==''] = 'UNKNOWN'
eon_data1['Aircraft Type'][eon_data1['Aircraft Type']=='UNKN'] = 'UNKNOWN'
eon_data1['Aircraft Type'][eon_data1['Aircraft Type']=='TYPE UNKNOWN'] = 'UNKNOWN'
eon_data1['Aircraft Type'][eon_data1['Aircraft Type']=='UNKNOWN TYPE'] = 'UNKNOWN'

eon_data1 = eon_data1.rename(columns={'Event Type':'Event_Type','Aircraft Type':'Aircraft_Type'})

eon_data1.insert(0,'Report_Type','EON')

# eon_data1.drop(['Title'], axis=1, inplace=True)
#Print the first 5 rows
eon_data1.head(5)

,Report_Type,Unique_Identifier,Title,Description,Event_Type,Date_Of_Occurrence,Event Time,Reported By,Received Date,Latitude,Longitude,City,State,Airport,Region,Reporting Facility,Aircraft_Type,Departure,Arrival,Diverted To,POB,Injury Type,Injuries Minor,Injuries Serious,Fatalities,Injuries Unknown,Uninjured,POB Unknown,Damage Type,MessageText,Remarks,N_Number
0,EON,554891,MEDICAL EMERGENCY - PASSENGER | ASA1305 | SAN ...,NaN,MEDICAL EMERGENCY - PASSENGER,2019-01-01,02:01,NaN,NaN,37.618805,-122.375420,SAN FRANCISCO,CA,SFO,AWP,MOR/SFO @,A320,NaN,SFO,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"SAN FRANCISCO, CA (SFO)",SFODECLARED A MEDICAL EMERGENCY FOR A 29 YEAR ...,ASA1305
1,EON,554892,GENERAL AVIATION OCCURRENCE | N9118C | SANTA R...,NaN,GENERAL AVIATION OCCURRENCE,2019-01-01,02:38,NaN,NaN,38.509693,-122.812889,SANTA ROSA,CA,STS,AWP,MOR/STS @ 1843 PST,C182,NaN,STS,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"SANTA ROSA, CA (STS)",STSLANDED WITHOUT INCIDENT AFTER REPORTING FAU...,9118C
2,EON,554894,GENERAL AVIATION OCCURRENCE | N613PJ | TETERBO...,NaN,GENERAL AVIATION OCCURRENCE,2019-01-01,03:15,NaN,NaN,40.850113,-74.060837,TETERBORO,NJ,TEB,AEA,TEB,CL60,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"TETERBORO, NJ (TEB)",DIVERTED TO TEB AND LANDED WITHOUT INCIDENT AF...,613PJ
3,EON,554895,"LASER EVENT | N4203P | PASCO, WA (PSC) 01-01-2...",NaN,LASER EVENT,2019-01-01,03:26,NaN,NaN,46.264694,-119.119026,PASCO,WA,PSC,ANM,MOR/PSC @ 1943 PST,C172,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"PASCO, WA (PSC)",WAS ILLUMINATED BY A GREEN LASER FROM THE 12 O...,4203P
4,EON,554908,AIR CARRIER OR TAXI OCCURRENCE | NKS8618 | DET...,NaN,AIR CARRIER OR TAXI OCCURRENCE,2019-01-01,11:25,NaN,NaN,42.212444,-83.353386,DETROIT,MI,DTW,AGL,DTW MOR,A320,DTW,DFW,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"DETROIT, MI (DTW)",DTW-DFW ABORTED TAKEOFF DUE TO A MALFUNCTION I...,NKS8618


In [10]:
#Dimensions of the Dataset
eon_data1.shape

(40465, 32)